In [87]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.linear_model import LinearRegression


import dask
import dask.dataframe as dd
import os
import scipy as sp
from tsfresh.feature_extraction import feature_calculators
from scipy import signal
import heapq
from scipy.signal import savgol_filter

from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


In [20]:
sampling_frequency = 4e6
number_sample = 36

In [3]:
%%time
train_df = pd.read_csv('train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

Wall time: 2min 46s


In [4]:
train_df.shape

(629145480, 2)

In [6]:
#numbers of rows 
rows = 150000
#numbers of segments of 150000 
segments = int(np.floor(train_df.shape[0] / rows))

In [7]:
segments

4194

In [22]:
#Defining 36 intervals, according with Carlos suggestions
subsegment=int(np.floor(rows/number_sample))
subsegment

4166

In [21]:
#creating a dataframe with no data for  x and y, dimension are segments x 1
X_train = pd.DataFrame(index=range(segments), dtype=np.float64)
y_train = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

In [23]:
#Definition of functions, to get the common features, and related to the spectrum shape.



def get_spectrum(input_signal):
    """
    Get a pandas Series with the fourier power spectum for a given signal segment.
    """
    input_signal = np.asarray(input_signal.values, dtype='float64')
    
    # Remove the mean  
    input_signal -= input_signal.mean()  
    
    # Estimate power spectral density using a periodogram.
    frequencies , power_spectrum = signal.periodogram(input_signal, sampling_frequency, scaling='spectrum')    
    
    # Run a running windows average of 10-points to smooth the signal.
    power_spectrum = pd.Series(power_spectrum, index=frequencies).rolling(window=10).mean()        
    
    return pd.Series(power_spectrum)

def get_segment_spectrum(segment_df):
    """
    Get the fourier power spectrum of a given segment.
    
    Returns the quake_time, frequencies, and power_spectrum
    """
    
    #quake_time =segment_df['time_to_failure'].values[-1]
    
    _power_spectrum = get_spectrum(segment_df['acoustic_data']).dropna() 
    
    # Keep only frequencies < 450khz (larger frequencies have a negligible contribution).
    _power_spectrum = _power_spectrum[_power_spectrum.index<450_000]
    
    # Keep one every 10 samples
    power_spectrum=_power_spectrum.values[::5]
    frequencies=_power_spectrum.index.values[::5]    
    
    return  frequencies, power_spectrum




In [24]:
# find maxs PSD 
#Nmax- how many max do you want?
def get_max_values(freq, power, Nmax):
    
    #finding peaks using inbuilt functions, condition on height
    #peaks_positive, _=signal.find_peaks(power_spectrum, height=0.1)
    #return indexes for power_spectrum
    peaks_positive, _=signal.find_peaks(power,  height=0.001)
    # find Nmax intensities values and order them
    max_intensities= np.array(heapq.nlargest(Nmax, power[peaks_positive]))
    #find Nmax intensities values and return their indexes
    index_frequencies= heapq.nlargest(Nmax, range(len(power[peaks_positive])), power[peaks_positive].__getitem__)
    #find max frequencies values
    max_frequencies=freq[peaks_positive][index_frequencies]
    #return numpy array
    return max_intensities, max_frequencies

#max_int, max_freq= get_max_values(_frequencies, power_spectrum)

In [25]:
# seg_id = segment
def create_features(segment, seg, Xt ):
    # to take kurtosis is necessary create a serie,
    x = pd.Series(seg['acoustic_data'].values)
    Xt.loc[segment, 'mean'] = x.mean()
    Xt.loc[segment, 'kurt'] = x.kurtosis()
    Xt.loc[segment, 'skew'] = x.skew()
    for i in range(number_sample):
        subpart=seg.iloc[i*subsegment:subsegment*i+subsegment]
        #quake_time, _frequencies, power_spectrum = get_segment_spectrum(subpart)
        _frequencies, power_spectrum = get_segment_spectrum(subpart)
        # normalize or not normalize!!!!!!!!!!!!!!!
        result = savgol_filter(power_spectrum/np.max(power_spectrum), 15, 4)
        #result = savgol_filter(power_spectrum, 15, 4)
        max_int, max_freq= get_max_values(_frequencies, result, 3)
        Xt.loc[segment, 'Int ' + str(i)+'-1'] = max_int[0]
        Xt.loc[segment, 'Int ' + str(i)+'-2'] = max_int[1]
        Xt.loc[segment, 'Int ' + str(i)+'-3'] = max_int[2]
        Xt.loc[segment, 'Freq ' + str(i)+'-1'] = max_freq[0]
        Xt.loc[segment, 'Freq ' + str(i)+'-2'] = max_freq[1]
        Xt.loc[segment, 'Freq ' + str(i)+'-3'] = max_freq[2]

    

In [26]:
for segment in tqdm_notebook(range(segments)):
    #done
    seg = train_df.iloc[segment*rows:segment*rows+rows]
   # .values  return the numpy representation of the given DataFrame      
    #done
    y_train.loc[segment, 'time_to_failure'] = seg['time_to_failure'].values[-1]
    create_features(segment, seg, X_train)
    
    

In [27]:
# cheking Dataframe
X_train.shape

(4194, 219)

In [28]:
X_train.head()

,mean,kurt,skew,Int 0-1,Int 0-2,Int 0-3,Freq 0-1,Freq 0-2,Freq 0-3,Int 1-1,...,Int 34-3,Freq 34-1,Freq 34-2,Freq 34-3,Int 35-1,Int 35-2,Int 35-3,Freq 35-1,Freq 35-2,Freq 35-3
0,4.884113,33.662481,-0.024061,0.732404,0.636964,0.149946,253480.556889,176668.266923,210273.643783,0.698115,...,0.145423,258281.325012,186269.803169,306289.006241,0.768794,0.327659,0.289910,253480.556889,215074.411906,181469.035046
1,4.725767,98.758517,0.390561,0.842336,0.567536,0.385974,248679.788766,176668.266923,215074.411906,0.814103,...,0.110755,258281.325012,195871.339414,296687.469995,0.844876,0.480550,0.439001,253480.556889,176668.266923,215074.411906
2,4.906393,33.555211,0.217391,0.618798,0.186305,0.147970,248679.788766,282285.165627,176668.266923,0.646589,...,0.174785,258281.325012,171867.498800,95055.208833,0.858529,0.613699,0.331679,267882.861258,195871.339414,143062.890062
3,4.902240,116.548172,0.757278,0.687366,0.224922,0.219061,267882.861258,224675.948152,239078.252520,0.698789,...,0.271580,167066.730677,258281.325012,85453.672588,0.723415,0.479196,0.215569,263082.093135,205472.875660,378300.528084
4,4.908720,52.977905,0.064531,0.852749,0.174800,0.131928,258281.325012,219875.180029,181469.035046,0.778168,...,0.519141,258281.325012,90254.440711,186269.803169,0.771947,0.430120,0.290565,258281.325012,181469.035046,383101.296207


In [29]:
X_train.tail()

,mean,kurt,skew,Int 0-1,Int 0-2,Int 0-3,Freq 0-1,Freq 0-2,Freq 0-3,Int 1-1,...,Int 34-3,Freq 34-1,Freq 34-2,Freq 34-3,Int 35-1,Int 35-2,Int 35-3,Freq 35-1,Freq 35-2,Freq 35-3
4189,4.446347,23.956541,-0.017274,0.625645,0.222914,0.216992,277484.397504,195871.339414,239078.252520,0.720294,...,0.395592,263082.093135,181469.035046,296687.469995,0.925899,0.110811,0.079016,263082.093135,191070.571291,219875.180029
4190,4.413793,28.942916,0.161954,0.684737,0.208356,0.201299,263082.093135,234277.484398,335093.614978,0.738498,...,0.252509,253480.556889,186269.803169,147863.658185,0.729890,0.704327,0.411912,99855.976956,253480.556889,181469.035046
4191,4.607200,9.986985,0.142006,0.805306,0.301255,0.296610,248679.788766,383101.296207,186269.803169,0.656493,...,0.202645,267882.861258,123859.817571,176668.266923,0.891720,0.853558,0.606971,176668.266923,248679.788766,133461.353817
4192,4.465280,60.777372,0.029714,0.835469,0.263071,0.092549,272683.629381,191070.571291,229476.716275,0.760832,...,0.386900,263082.093135,311089.774364,143062.890062,0.648395,0.097606,0.086182,243879.020643,200672.107537,167066.730677
4193,4.518400,15.873626,-0.123669,0.864876,0.391663,0.272207,248679.788766,176668.266923,133461.353817,0.891709,...,0.163421,176668.266923,263082.093135,138262.121940,0.748400,0.716555,0.707615,267882.861258,282285.165627,253480.556889


In [88]:
#X_train.isnull().sum()

In [89]:
#y_train.head()

In [32]:
#scaling data
scaler = StandardScaler()
scaler.fit(X_train)
scaled_train_X = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)

In [33]:
#scaled_train_X.head(10)

In [34]:
#scaled_train_X.isnull().sum()

# CROSS VALIDATION

In [82]:
# create training and testing vars
X_tra, X_te, y_tra, y_te = train_test_split(scaled_train_X, y_train.values.flatten(), test_size=0.2)
#checking size data 
print(X_tra.shape, y_tra.shape)
print(X_te.shape, y_te.shape)

(3355, 219) (3355,)
(839, 219) (839,)


In [91]:
#fit model
#svm = NuSVR()
svm=NuSVR(nu=0.5, kernel='rbf', gamma='scale',max_iter=-1)
model=svm.fit(X_tra, y_tra)
y_preda = svm.predict(X_te)


In [92]:
scores = mean_absolute_error(y_preda, y_te)
print(f'Score:', scores)

Score: 2.340600391681378


# SVR- considering  complete data

In [99]:
#svm = NuSVR(nu=0.8, kernel='poly', max_iter=-1)
svm = NuSVR(nu=0.8, kernel='rbf', gamma='scale',max_iter=-1)
svm.fit(scaled_train_X, y_train.values.flatten())
y_pred = svm.predict(scaled_train_X)

In [100]:
score = mean_absolute_error(y_train.values.flatten(), y_pred)
print(f'Score: {score:0.3f}')

Score: 1.752


In [102]:
#creating submission file

submission = pd.read_csv('sample_submission.csv', index_col='seg_id')
test_X = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [103]:
submission.shape, test_X.shape

((2624, 1), (2624, 219))

In [40]:
#test_X.head

In [104]:
## DOUBLE CHECK
#PATH='C:\\Users\\John\\Desktop\\Data Books\\LanL Earthquake competition\\test'
PATH='C:\\Users\\John\\Desktop\\Data Books\\LanL Earthquake competition\\test\\'
#PATH='C:\\Users\\John\\Desktop\\Data Books\\LanL Earthquake competition\\test'

for seg_id in tqdm_notebook(test_X.index):
    seg = pd.read_csv( PATH + seg_id + '.csv')
    create_features(seg_id, seg, test_X)

In [ ]:
#test_X.shape
        

In [ ]:
#test_X.tail()

In [105]:
scaled_test_X = pd.DataFrame(scaler.transform(test_X), columns=test_X.columns)

In [106]:
scaled_test_X.shape

(2624, 219)

In [107]:
submission['time_to_failure'] = svm.predict(scaled_test_X)
submission.to_csv('submissionb.csv')